In [1]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken
from dotenv import load_dotenv

In [2]:
# Azure
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

KAGGLE_USERNAME = os.getenv("KAGGLE_USERNAME")
KAGGLE_KEY = os.getenv("KAGGLE_KEY")

server = os.getenv("DATABASESERVER")
database = os.getenv("DATABASE") 
username = os.getenv("DATABASEUSERNAME")  
password = os.getenv("DATABASEPASSWORD") 

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

load_dotenv()

True

In [14]:
!kaggle datasets download -d shivamb/netflix-shows


  0%|          | 0.00/1.34M [00:00<?, ?B/s]
 75%|#######4  | 1.00M/1.34M [00:02<00:00, 518kB/s]
100%|##########| 1.34M/1.34M [00:02<00:00, 615kB/s]
100%|##########| 1.34M/1.34M [00:02<00:00, 590kB/s]


In [16]:
#Unzip the downloaded file
import zipfile
with zipfile.ZipFile("netflix-shows.zip", 'r') as zip_ref:
    zip_ref.extractall("netflix_dataset")


In [5]:
data = pd.read_csv("netflix_dataset/netflix_titles.csv")
data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,25-Sep-21,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,24-Sep-21,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,24-Sep-21,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
1994,s1995,TV Show,Sing On!,NaN,Tituss Burgess,United States,16-Sep-20,2020,TV-PG,1 Season,Reality TV,"In this fun, fast-paced music contest hosted b..."
1995,s1996,Movie,The Devil All The Time,Antonio Campos,"Tom Holland, Bill Skarsgård, Riley Keough, Rob...",United States,16-Sep-20,2020,R,139 min,"Dramas, Thrillers",Sinister characters converge around a young ma...
1996,s1997,Movie,The Paramedic,Carles Torras,"Mario Casas, Déborah François, Guillermo Pfeni...",Spain,16-Sep-20,2020,TV-MA,94 min,"Dramas, International Movies, Thrillers",Unable to face his new reality in a wheelchair...
1997,s1998,Movie,The Take,James Watkins,"Idris Elba, Richard Madden, Charlotte Le Bon, ...","United Kingdom, France, United States, Belgium...",16-Sep-20,2016,R,92 min,"Action & Adventure, International Movies","After a mix-up lands him in hot water, a skill..."


In [6]:
data_content = data[['title', 'description']]
data_content

,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...
...,...,...
1994,Sing On!,"In this fun, fast-paced music contest hosted b..."
1995,The Devil All The Time,Sinister characters converge around a young ma...
1996,The Paramedic,Unable to face his new reality in a wheelchair...
1997,The Take,"After a mix-up lands him in hot water, a skill..."


In [17]:
import re

pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

data_content['title']= data_content["title"].apply(lambda x : normalize_text(x))
data_content

,title,description,ada_v2,similarities
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm...","[0.012696618214249611, 0.001742478576488793, -...",0.812542
1,Blood & Water,"After crossing paths at a party, a Cape Town t...","[0.015194851905107498, -0.015367219224572182, ...",1.000000
2,Ganglands,To protect his family from a powerful drug lor...,"[-0.004515422508120537, -0.008777394890785217,...",0.794925
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo...","[0.00742305675521493, -0.017011739313602448, 0...",0.781428
4,Kota Factory,In a city of coaching centers known to train I...,"[0.015928281471133232, -0.01370211225003004, 0...",0.813568
...,...,...,...,...
1994,Sing On!,"In this fun, fast-paced music contest hosted b...","[-0.007583182770758867, -0.01742655225098133, ...",0.750352
1995,The Devil All The Time,Sinister characters converge around a young ma...,"[0.00908596906810999, -0.022806042805314064, 0...",0.802324
1996,The Paramedic,Unable to face his new reality in a wheelchair...,"[0.005674256477504969, -0.016323383897542953, ...",0.803520
1997,The Take,"After a mix-up lands him in hot water, a skill...","[-0.016791721805930138, -0.01369066908955574, ...",0.831377


In [8]:
# Apply the function to each row in the dataset and store the embeddings in a new column

data_content['ada_v2'] = data_content["description"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # Specify the file path for saving the CSV file


In [9]:
data_content

,title,description,ada_v2
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm...","[0.012696618214249611, 0.001742478576488793, -..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t...","[0.015194851905107498, -0.015367219224572182, ..."
2,Ganglands,To protect his family from a powerful drug lor...,"[-0.004515422508120537, -0.008777394890785217,..."
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo...","[0.00742305675521493, -0.017011739313602448, 0..."
4,Kota Factory,In a city of coaching centers known to train I...,"[0.015928281471133232, -0.01370211225003004, 0..."
...,...,...,...
1994,Sing On!,"In this fun, fast-paced music contest hosted b...","[-0.007583182770758867, -0.01742655225098133, ..."
1995,The Devil All The Time,Sinister characters converge around a young ma...,"[0.00908596906810999, -0.022806042805314064, 0..."
1996,The Paramedic,Unable to face his new reality in a wheelchair...,"[0.005674256477504969, -0.016323383897542953, ..."
1997,The Take,"After a mix-up lands him in hot water, a skill...","[-0.016791721805930138, -0.01369066908955574, ..."


In [18]:
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

title = "Blood & Water"
description = data_content.loc[data_content['title'] == title, "description"].iloc[0]
res = search_docs(data_content, description, top_n=15)

,title,description,ada_v2,similarities
1,Blood & Water,"After crossing paths at a party, a Cape Town t...","[0.015194851905107498, -0.015367219224572182, ...",1.000000
108,Dive Club,"On the shores of Cape Mercy, a skillful group ...","[0.01811172254383564, -0.027001969516277313, 0...",0.874606
1409,The Vanished,When their daughter disappears during a family...,"[0.029839424416422844, 0.006644036620855331, 0...",0.870062
613,Voiceless,A bright and promising young girl gets abducte...,"[-0.0192096009850502, -0.03516734018921852, -0...",0.868562
640,Jiva!,"A talented street dancer from Umlazi, Durban m...","[-0.006459933239966631, -0.02330109290778637, ...",0.864880
1067,The Underclass,"After getting dropped from an elite program, a...","[-0.010393356904387474, -0.021632760763168335,...",0.863741
335,Freedomland,When a woman reports that her infant son was k...,"[0.016996609047055244, -0.024279022589325905, ...",0.863431
1403,Double Dad,"While her mom is away, a teen sneaks out of th...","[0.02948031574487686, -0.011788301169872284, -...",0.862672
711,Security,After the assault of a young woman in their se...,"[0.004381557460874319, -0.009267902001738548, ...",0.861973
352,The Edge of Seventeen,A troubled teenager finds high school life eve...,"[-0.0023346757516264915, -0.023263489827513695...",0.860551


,title,description,ada_v2,similarities
1,Blood & Water,"After crossing paths at a party, a Cape Town t...","[0.015194851905107498, -0.015367219224572182, ...",1.000000
108,Dive Club,"On the shores of Cape Mercy, a skillful group ...","[0.01811172254383564, -0.027001969516277313, 0...",0.874606
1409,The Vanished,When their daughter disappears during a family...,"[0.029839424416422844, 0.006644036620855331, 0...",0.870062
613,Voiceless,A bright and promising young girl gets abducte...,"[-0.0192096009850502, -0.03516734018921852, -0...",0.868562
640,Jiva!,"A talented street dancer from Umlazi, Durban m...","[-0.006459933239966631, -0.02330109290778637, ...",0.864880
1067,The Underclass,"After getting dropped from an elite program, a...","[-0.010393356904387474, -0.021632760763168335,...",0.863741
335,Freedomland,When a woman reports that her infant son was k...,"[0.016996609047055244, -0.024279022589325905, ...",0.863431
1403,Double Dad,"While her mom is away, a teen sneaks out of th...","[0.02948031574487686, -0.011788301169872284, -...",0.862672
711,Security,After the assault of a young woman in their se...,"[0.004381557460874319, -0.009267902001738548, ...",0.861973
352,The Edge of Seventeen,A troubled teenager finds high school life eve...,"[-0.0023346757516264915, -0.023263489827513695...",0.860551
